In [ ]:
run = "resnet 18 high dropout"
run_title = r"/content/runs/{0}".format(run)

In [ ]:
from os import getcwd

getcwd()

'/content'

In [ ]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

import os
import zipfile

file_prefix = ''
if is_colab():
    run_title = "/content/drive/MyDrive/ringo/runs/" + run
    from google.colab import drive
    drive.mount('/content/drive')
    # Specify the path to the folder where the zip will be extracted
    extracted_folder_path = 'ringo'
    # Check if the folder already exists
    if not os.path.exists(extracted_folder_path):
        with zipfile.ZipFile('/content/drive/MyDrive/ringo/ringo.zip', 'r') as zip_ref:
            zip_ref.extractall(extracted_folder_path)
    # Optionally, you can unmount the drive once you are done with file operations
    # drive.flush_and_unmount()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil

# Define the source and destination paths
source_path = '/content/drive/MyDrive/ringo/ringo_dataset.py'
destination_path = 'ringo_dataset.py'

# Check if the destination file already exists
if not os.path.exists(destination_path):
    # Copy the file
    shutil.copy(source_path, destination_path)
    print(f'File copied from {source_path} to {destination_path}')
else:
    print(f'The file {destination_path} already exists. No action taken.')

The file ringo_dataset.py already exists. No action taken.


In [ ]:
os.chdir('/content')
getcwd()

'/content'

In [ ]:
import torch
from torch import nn
import torchvision

In [ ]:
from ringo_dataset import RingoDataset
from torch.utils.tensorboard import SummaryWriter

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    run_title = run_title + " (cuda)"
    torch.cuda.empty_cache()
    print("We've got a graphics card!")
else:
    device = torch.device("cpu")
    print("looks like we've got a wimpy little cpu")
writer = SummaryWriter(run_title)

looks like we've got a wimpy little cpu


In [ ]:
# Load the pre-defined model
model = torchvision.models.resnet18()
model = model.to(device)

# Modify the classifier
# Add ReLU activation followed by a linear layer from 1000 to 1
model.fc = nn.Sequential(
    nn.Dropout(0.9),
    model.fc,  # Original classifier layers
    nn.ReLU(),           # Additional ReLU activation
    nn.Linear(1000, 1)   # Additional Linear layer from 1000 to 1
).to(device)

# model = nn.Sequential(nn.Dropout(0.0).to(device), model)

# model.load_state_dict(torch.load("autoaim_resnet 18 dropout 0.5 2 (no dropout now).pth", map_location = device))

# Create a test input
# expects a 3 channel image, but the size can be flexible
test_input = torch.randn(1, 3, 200, 200).to(device)  # Batch size of 1

with torch.no_grad():
    # Forward pass through the model
    output = model(test_input)


In [ ]:
learning_rate = 1e-3
weight_decay = 0.0
writer.add_text("info", f"learning rate: {learning_rate}")

optimizer = torch.optim.AdamW(lr = learning_rate, params = model.parameters(), weight_decay = weight_decay)

criterion = torch.nn.MSELoss()

In [ ]:
batch_size = 100

In [ ]:
# prompt: print everything in the current working directory in python

import os

# Get the current working directory
cwd = os.getcwd()

# List all files and directories in the current working directory
files_and_directories = os.listdir(cwd)

# Print each file and directory
for file_or_directory in files_and_directories:
    print(file_or_directory)


.config
ringo_dataset.py
drive
__pycache__
ringo
sample_data


In [ ]:
training_dataset = RingoDataset(r"ringo/training", aiming = True, image_limit = None)
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size = batch_size, shuffle = True)
testing_dataset = RingoDataset(r"ringo/testing", aiming = True, image_limit = None)
testing_loader = torch.utils.data.DataLoader(testing_dataset, batch_size = batch_size, shuffle = True)

Loading images...
number of samples: 2959


100%|██████████| 2959/2959 [00:24<00:00, 119.61it/s]


Loading images...
number of samples: 735


100%|██████████| 735/735 [00:05<00:00, 125.66it/s]


In [ ]:
# test a sample to make sure that the model's shapes work out
some_image_sample = training_dataset[0][0].to(device)
model(training_dataset[0][0].unsqueeze(0))

tensor([[-0.3426]], grad_fn=<AddmmBackward0>)

In [ ]:
def test(model, criterion, testing_loader, epoch):
    with torch.no_grad():
        total_loss = 0
        total_batches = 0
        correct = 0
        almost_correct = 0
        not_really_correct = 0
        wrong = 0
        samples = 0
        for batch in testing_loader:
            total_batches += 1
            images, stamp_types, inside_diameters, y_offsets = batch
            y_offsets = y_offsets.unsqueeze(1).to(device) # TODO: also fix this? >.> (is it because there's no channel dim?)
            output = model(images)
            # print(output, y_offsets)
            for some_output, some_y_offset in zip(output, y_offsets):
                samples += 1
                difference = torch.abs(torch.round(some_output[0], decimals=1) - some_y_offset[0])
                if difference < 0.1:
                    correct += 1
                if difference < 0.2:
                    almost_correct += 1
                if difference < 0.3:
                    not_really_correct += 1
                else:
                    wrong += 1
            loss = criterion(output, y_offsets)
            total_loss += loss
        writer.add_scalar("accuracy/correct within 0.1", correct / samples, epoch)
        writer.add_scalar("accuracy/corrent within 0.2", almost_correct / samples, epoch)
        writer.add_scalar("accuracy/correct within 0.3", not_really_correct / samples, epoch)
        writer.add_scalar("accuracy/more than 0.3", wrong / samples, epoch)
    return total_loss / total_batches

In [ ]:
# prompt: display tensorboard

%load_ext tensorboard
%tensorboard --logdir=/content/drive/MyDrive/ringo/runs


In [ ]:
epochs = 100

for epoch in range(epochs):
    print("welcome to epoch:", epoch + 1)
    batch_count = 0
    total_loss = 0
    for i, batch in enumerate(training_loader):
        # print("Batch:", i + 1)
        batch_count += 1
        optimizer.zero_grad()
        image, stamp_type, inside_diameter, y_offset = batch
        y_offset = y_offset.unsqueeze(1) # fix this... >.>
        output = model(image)
        # print(output.shape, y_offset.shape)
        # print("outputs vs. expected_outputs:")
        loss = criterion(output, y_offset)
        total_loss += loss
        loss.backward()
        optimizer.step()
    average_training_loss = total_loss / batch_count
    writer.add_scalar("loss/training", average_training_loss, epoch)
    test_loss = test(model, criterion, testing_loader, epoch)
    writer.add_scalar("loss/testing", test_loss, epoch)
    writer.flush()
writer.close()

welcome to epoch: 1


In [ ]:


torch.save(model.state_dict(), f"/content/drive/MyDrive/ringo/autoaim_{run}.pt")

torch.save(model, f"/content/drive/MyDrive/ringo/autoaim_{run}_full_model.pth")

In [ ]:
writer.close()